# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saint-pierre,-21.3393,55.4781,18.82,77,100,2.57,RE,1722645018
1,1,puerto ayora,-0.7393,-90.3518,19.90,78,41,5.27,EC,1722645019
2,2,grytviken,-54.2811,-36.5092,0.04,88,100,5.98,GS,1722645020
3,3,aginskoye,51.1000,114.5300,15.45,96,100,4.30,RU,1722645021
4,4,palangkaraya,-2.2000,113.8333,27.27,81,84,0.68,ID,1722645023


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [73]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha=0.5
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
#Wind speed less than 4.5 m/s
#Zero cloudiness
ideal_city_conditions_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &
    (city_data_df['Max Temp'] > 21) &
    (city_data_df['Wind Speed'] < 4.5) &
    (city_data_df['Cloudiness'] == 0) 
]
# Drop any rows with null values
ideal_cities_df = ideal_city_conditions_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,taman',45.2117,36.7161,24.60,71,0,3.07,RU,1722645080
97,97,presque isle,46.6811,-68.0159,23.32,87,0,0.00,US,1722645136
168,168,pacific grove,36.6177,-121.9166,22.67,90,0,3.09,US,1722645051
202,202,diamantino,-14.4086,-56.4461,24.40,35,0,2.99,BR,1722645264
225,225,yinchuan,38.4681,106.2731,22.54,67,0,2.25,CN,1722645292
300,300,princeville,45.7668,-61.2985,21.84,92,0,1.03,CA,1722645386
309,309,ribeira brava,32.6500,-17.0667,24.35,71,0,1.31,PT,1722645396
347,347,muynoq,43.7683,59.0214,25.03,33,0,3.34,UZ,1722645447
357,357,feres,40.8956,26.1700,21.69,64,0,0.00,GR,1722645458
373,373,sao felix do xingu,-6.6447,-51.9950,24.79,37,0,1.18,BR,1722645478


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
52,taman',RU,45.2117,36.7161,71,
97,presque isle,US,46.6811,-68.0159,87,
168,pacific grove,US,36.6177,-121.9166,90,
202,diamantino,BR,-14.4086,-56.4461,35,
225,yinchuan,CN,38.4681,106.2731,67,
300,princeville,CA,45.7668,-61.2985,92,
309,ribeira brava,PT,32.6500,-17.0667,71,
347,muynoq,UZ,43.7683,59.0214,33,
357,feres,GR,40.8956,26.1700,64,
373,sao felix do xingu,BR,-6.6447,-51.9950,37,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories':'accommodation.hotel',
    'apiKey': geoapify_key,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taman' - nearest hotel: Волна
presque isle - nearest hotel: The Northeastland Hotel
pacific grove - nearest hotel: Pacific Grove Inn
diamantino - nearest hotel: Hotel kayaby
yinchuan - nearest hotel: Yinchuan hotel
princeville - nearest hotel: No hotel found
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
muynoq - nearest hotel: Lala
feres - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: Hotel Terraço
sochi - nearest hotel: Марианна
campoverde - nearest hotel: No hotel found
melfort - nearest hotel: No hotel found
manzano - nearest hotel: Elliot Hotel
laojunmiao - nearest hotel: No hotel found
marquette - nearest hotel: No hotel found
lagoa - nearest hotel: Casa da Madrinha
misratah - nearest hotel: الفندق السياحى


,City,Country,Lat,Lng,Humidity,Hotel Name
52,taman',RU,45.2117,36.7161,71,Волна
97,presque isle,US,46.6811,-68.0159,87,The Northeastland Hotel
168,pacific grove,US,36.6177,-121.9166,90,Pacific Grove Inn
202,diamantino,BR,-14.4086,-56.4461,35,Hotel kayaby
225,yinchuan,CN,38.4681,106.2731,67,Yinchuan hotel
300,princeville,CA,45.7668,-61.2985,92,No hotel found
309,ribeira brava,PT,32.6500,-17.0667,71,Cheerfulway Bravamar Hotel
347,muynoq,UZ,43.7683,59.0214,33,Lala
357,feres,GR,40.8956,26.1700,64,No hotel found
373,sao felix do xingu,BR,-6.6447,-51.9950,37,Hotel Terraço


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [79]:
%%capture --no-display

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols='all',
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "Country",
    alpha = .5
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,index,City,Hotel Name)